In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1735793,2021-10-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1735794,2021-10-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1735795,2021-10-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1735796,2021-10-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
45255,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
45257,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
45259,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
45261,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
45263,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1057428,2021-10-10,Arkansas,Arkansas,5001,3431,58.00,5000,Arkansas,AR,Arkansas,State,3017804
1057430,2021-10-11,Arkansas,Arkansas,5001,3433,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1057432,2021-10-12,Arkansas,Arkansas,5001,3437,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1057434,2021-10-13,Arkansas,Arkansas,5001,3439,59.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1735793,2021-10-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1735794,2021-10-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1735795,2021-10-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1735796,2021-10-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-10-14') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
632,2021-10-14,Snohomish,Washington,61965,727.00,53061,WA,County,822083,7537.56,88.43
1262,2021-10-14,Cook,Illinois,630542,11712.00,17031,IL,County,5150233,12242.98,227.41
1891,2021-10-14,Orange,California,322497,5513.00,6059,CA,County,3175692,10155.17,173.60
2519,2021-10-14,Maricopa,Arizona,715923,11781.00,4013,AZ,County,4485414,15961.14,262.65
3147,2021-10-14,Los Angeles,California,1475222,26379.00,6037,CA,County,10039107,14694.75,262.76
...,...,...,...,...,...,...,...,...,...,...,...
1734453,2021-10-14,Wheeler,Oregon,99,1.00,41069,OR,County,1332,7432.43,75.08
1734820,2021-10-14,King,Texas,16,0.00,48269,TX,County,272,5882.35,0.00
1735156,2021-10-14,Esmeralda,Nevada,58,2.00,32009,NV,County,873,6643.76,229.10
1735488,2021-10-14,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
632,2021-10-14,Snohomish,Washington,61965,727.00,53061,WA,County,822083,7537.56,88.43,88.43,7537.56
1262,2021-10-14,Cook,Illinois,630542,11712.00,17031,IL,County,5150233,12242.98,227.41,227.41,12242.98
1891,2021-10-14,Orange,California,322497,5513.00,6059,CA,County,3175692,10155.17,173.60,173.60,10155.17
2519,2021-10-14,Maricopa,Arizona,715923,11781.00,4013,AZ,County,4485414,15961.14,262.65,262.65,15961.14
3147,2021-10-14,Los Angeles,California,1475222,26379.00,6037,CA,County,10039107,14694.75,262.76,262.76,14694.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734453,2021-10-14,Wheeler,Oregon,99,1.00,41069,OR,County,1332,7432.43,75.08,75.08,7432.43
1734820,2021-10-14,King,Texas,16,0.00,48269,TX,County,272,5882.35,0.00,0.00,5882.35
1735156,2021-10-14,Esmeralda,Nevada,58,2.00,32009,NV,County,873,6643.76,229.10,229.10,6643.76
1735488,2021-10-14,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1106023,2021-10-14,Hale,Alabama,3085,86.00,1065,AL,County,14651,21056.58,586.99,586.99,21056.58,1
1204643,2021-10-14,Clarke,Alabama,4726,81.00,1025,AL,County,23622,20006.77,342.90,342.90,20006.77,2
312428,2021-10-14,Calhoun,Alabama,22095,468.00,1015,AL,County,113605,19448.97,411.95,411.95,19448.97,3
765650,2021-10-14,Etowah,Alabama,19727,496.00,1055,AL,County,102268,19289.51,485.00,485.00,19289.51,4
436141,2021-10-14,Jackson,Alabama,9760,177.00,1071,AL,County,51626,18905.20,342.85,342.85,18905.20,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648740,2021-10-14,Weston,Wyoming,978,8.00,56045,WY,County,6927,14118.67,115.49,115.49,14118.67,19
930537,2021-10-14,Johnson,Wyoming,1136,15.00,56019,WY,County,8445,13451.75,177.62,177.62,13451.75,20
1055741,2021-10-14,Sublette,Wyoming,1306,13.00,56035,WY,County,9831,13284.51,132.23,132.23,13284.51,21
1314040,2021-10-14,Lincoln,Wyoming,2576,18.00,56023,WY,County,19830,12990.42,90.77,90.77,12990.42,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
864081,2021-10-14,Lowndes,Alabama,1730,61.00,1085,AL,County,9726,17787.37,627.18,627.18,17787.37,17,1
1106023,2021-10-14,Hale,Alabama,3085,86.00,1065,AL,County,14651,21056.58,586.99,586.99,21056.58,1,2
365993,2021-10-14,Walker,Alabama,11055,363.00,1127,AL,County,63521,17403.69,571.46,571.46,17403.69,28,3
862945,2021-10-14,Crenshaw,Alabama,2430,75.00,1041,AL,County,13772,17644.50,544.58,544.58,17644.50,22,4
932238,2021-10-14,Greene,Alabama,1253,44.00,1063,AL,County,8111,15448.16,542.47,542.47,15448.16,48,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281000,2021-10-14,Uinta,Wyoming,3705,26.00,56041,WY,County,20226,18318.01,128.55,128.55,18318.01,5,19
1648740,2021-10-14,Weston,Wyoming,978,8.00,56045,WY,County,6927,14118.67,115.49,115.49,14118.67,19,20
1314040,2021-10-14,Lincoln,Wyoming,2576,18.00,56023,WY,County,19830,12990.42,90.77,90.77,12990.42,22,21
860673,2021-10-14,Albany,Wyoming,6055,31.00,56001,WY,County,38880,15573.56,79.73,79.73,15573.56,14,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,88.43,7537.56,27,29
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,88.43,7537.56,27,29
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,88.43,7537.56,27,29
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,88.43,7537.56,27,29
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,88.43,7537.56,27,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731768,2021-10-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1731769,2021-10-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1731770,2021-10-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1731771,2021-10-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1101435,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,586.99,21056.58,2,1,1.00,nan
1101436,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,586.99,21056.58,2,1,0.00,nan
1101437,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,586.99,21056.58,2,1,0.00,nan
1101438,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,586.99,21056.58,2,1,0.00,nan
1101439,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,586.99,21056.58,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336284,2021-10-10,Crook,Wyoming,816,17.00,56011,WY,County,7584,10759.49,224.16,237.34,11010.02,7,23,0.00,5.14
1336285,2021-10-11,Crook,Wyoming,820,17.00,56011,WY,County,7584,10812.24,224.16,237.34,11010.02,7,23,4.00,4.64
1336286,2021-10-12,Crook,Wyoming,828,18.00,56011,WY,County,7584,10917.72,237.34,237.34,11010.02,7,23,8.00,4.71
1336287,2021-10-13,Crook,Wyoming,834,18.00,56011,WY,County,7584,10996.84,237.34,237.34,11010.02,7,23,6.00,4.93


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
860621,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17787.37,1,17,1.00,8.93,0.00,nan
860622,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17787.37,1,17,0.00,8.50,0.00,nan
860623,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17787.37,1,17,0.00,8.00,0.00,nan
860624,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17787.37,1,17,0.00,7.57,0.00,nan
860625,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17787.37,1,17,0.00,7.07,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361940,2021-10-10,Teton,Wyoming,4984,13.00,56039,WY,County,23464,21241.05,55.40,55.40,21475.45,23,1,0.00,7.21,0.00,0.07
361941,2021-10-11,Teton,Wyoming,5000,13.00,56039,WY,County,23464,21309.24,55.40,55.40,21475.45,23,1,16.00,7.21,0.00,0.07
361942,2021-10-12,Teton,Wyoming,5006,13.00,56039,WY,County,23464,21334.81,55.40,55.40,21475.45,23,1,6.00,6.50,0.00,0.00
361943,2021-10-13,Teton,Wyoming,5018,13.00,56039,WY,County,23464,21385.95,55.40,55.40,21475.45,23,1,12.00,6.57,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-10-14') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
437261,2021-10-14,Imperial,California,35591,761.00,6025,CA,County,181215,19640.21,419.94,419.94,19640.21,1,3,88.00,56.71,0.00,0.36
3147,2021-10-14,Los Angeles,California,1475222,26379.00,6037,CA,County,10039107,14694.75,262.76,262.76,14694.75,2,12,1108.00,1095.07,17.00,19.50
217069,2021-10-14,San Bernardino,California,355848,5562.00,6071,CA,County,2180085,16322.67,255.13,255.13,16322.67,3,5,182.00,316.07,-1.00,1.21
96749,2021-10-14,Stanislaus,California,84923,1310.00,6099,CA,County,550660,15422.04,237.90,237.90,15422.04,4,7,214.00,167.71,8.00,1.93
76895,2021-10-14,San Joaquin,California,101643,1702.00,6077,CA,County,762148,13336.39,223.32,223.32,13336.39,5,18,182.00,151.14,0.00,3.43
767878,2021-10-14,Inyo,California,1942,39.00,6027,CA,County,18039,10765.56,216.20,216.20,10765.56,6,28,24.00,12.50,0.00,0.00
695505,2021-10-14,Merced,California,42035,582.00,6047,CA,County,277680,15137.93,209.59,209.59,15137.93,7,9,123.00,89.64,1.00,1.79
44667,2021-10-14,Riverside,California,368953,5080.00,6065,CA,County,2470546,14934.07,205.62,205.62,14934.07,8,10,363.00,405.14,12.00,6.14
43493,2021-10-14,Fresno,California,143745,2048.00,6019,CA,County,999101,14387.43,204.98,204.98,14387.43,9,13,380.00,313.71,6.00,6.50
932180,2021-10-14,Kings,California,32197,312.00,6031,CA,County,152940,21052.05,204.00,204.00,21052.05,10,2,104.00,83.50,0.00,0.93


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1637098,2021-10-14,Lassen,California,7437,45.00,6035,CA,County,30573,24325.39,147.19,147.19,24325.39,22,1,19.00,15.86,2.00,0.50
932180,2021-10-14,Kings,California,32197,312.00,6031,CA,County,152940,21052.05,204.00,204.00,21052.05,10,2,104.00,83.50,0.00,0.93
437261,2021-10-14,Imperial,California,35591,761.00,6025,CA,County,181215,19640.21,419.94,419.94,19640.21,1,3,88.00,56.71,0.00,0.36
120066,2021-10-14,Tulare,California,78753,945.00,6107,CA,County,466195,16892.72,202.70,202.70,16892.72,11,4,407.00,217.29,1.00,2.64
217069,2021-10-14,San Bernardino,California,355848,5562.00,6071,CA,County,2180085,16322.67,255.13,255.13,16322.67,3,5,182.00,316.07,-1.00,1.21
273177,2021-10-14,Kern,California,143716,1600.00,6029,CA,County,900202,15964.86,177.74,177.74,15964.86,15,6,444.00,335.36,3.00,4.00
96749,2021-10-14,Stanislaus,California,84923,1310.00,6099,CA,County,550660,15422.04,237.90,237.90,15422.04,4,7,214.00,167.71,8.00,1.93
996247,2021-10-14,Glenn,California,4299,40.00,6021,CA,County,28393,15141.06,140.88,140.88,15141.06,25,8,27.00,10.57,0.00,0.21
695505,2021-10-14,Merced,California,42035,582.00,6047,CA,County,277680,15137.93,209.59,209.59,15137.93,7,9,123.00,89.64,1.00,1.79
44667,2021-10-14,Riverside,California,368953,5080.00,6065,CA,County,2470546,14934.07,205.62,205.62,14934.07,8,10,363.00,405.14,12.00,6.14


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
437261,2021-10-14,Imperial,California,35591,761.00,6025,CA,County,181215,19640.21,419.94,419.94,19640.21,1,3,88.00,56.71,0.00,0.36
3147,2021-10-14,Los Angeles,California,1475222,26379.00,6037,CA,County,10039107,14694.75,262.76,262.76,14694.75,2,12,1108.00,1095.07,17.00,19.50
217069,2021-10-14,San Bernardino,California,355848,5562.00,6071,CA,County,2180085,16322.67,255.13,255.13,16322.67,3,5,182.00,316.07,-1.00,1.21
96749,2021-10-14,Stanislaus,California,84923,1310.00,6099,CA,County,550660,15422.04,237.90,237.90,15422.04,4,7,214.00,167.71,8.00,1.93
76895,2021-10-14,San Joaquin,California,101643,1702.00,6077,CA,County,762148,13336.39,223.32,223.32,13336.39,5,18,182.00,151.14,0.00,3.43
767878,2021-10-14,Inyo,California,1942,39.00,6027,CA,County,18039,10765.56,216.20,216.20,10765.56,6,28,24.00,12.50,0.00,0.00
695505,2021-10-14,Merced,California,42035,582.00,6047,CA,County,277680,15137.93,209.59,209.59,15137.93,7,9,123.00,89.64,1.00,1.79
44667,2021-10-14,Riverside,California,368953,5080.00,6065,CA,County,2470546,14934.07,205.62,205.62,14934.07,8,10,363.00,405.14,12.00,6.14
43493,2021-10-14,Fresno,California,143745,2048.00,6019,CA,County,999101,14387.43,204.98,204.98,14387.43,9,13,380.00,313.71,6.00,6.50
932180,2021-10-14,Kings,California,32197,312.00,6031,CA,County,152940,21052.05,204.00,204.00,21052.05,10,2,104.00,83.50,0.00,0.93


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,73.50
1,03/21/20,Imperial,4,2.21,3,0.00,73.50
2,03/22/20,Imperial,4,2.21,3,0.00,73.50
3,03/23/20,Imperial,4,2.21,3,0.00,53.21
4,03/24/20,Imperial,9,4.97,3,5.00,47.93
...,...,...,...,...,...,...,...
5690,10/10/21,Glenn,4228,14890.99,8,0.00,10.93
5691,10/11/21,Glenn,4248,14961.43,8,20.00,9.64
5692,10/12/21,Glenn,4259,15000.18,8,11.00,9.93
5693,10/13/21,Glenn,4272,15045.96,8,13.00,9.71


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
5823,10/10/21,Kings,308.00,201.39,10,0.00,1.07
5824,10/11/21,Kings,308.00,201.39,10,0.00,1.07
5825,10/12/21,Kings,309.00,202.04,10,1.00,0.86
5826,10/13/21,Kings,312.00,204.00,10,3.00,1.14


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)